In [1]:
import sys
import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb

from catboost import CatBoostRegressor

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

from sklearn.ensemble import GradientBoostingRegressor

from sklearn.kernel_ridge import KernelRidge

In [2]:
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
train   = pd.read_csv('train_cleaning.csv')
test    = pd.read_csv('test_cleaning.csv')
y_train = pd.read_csv('y_train.csv')

In [4]:
y_train = y_train.values.ravel()

In [5]:
X_train, X_val, Y_train, Y_val = train_test_split(train, y_train, test_size = 0.2, random_state=42)

In [6]:
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train)  
    rmse= np.sqrt(-cross_val_score(model, train, y_train, cv = kf, scoring="neg_mean_squared_error"))
    return(rmse)

### Kernel Ridge

In [7]:
KR = make_pipeline(RobustScaler(), KernelRidge(
         alpha= 0.9826,
         kernel= 'polynomial',
         degree= 2,
         coef0= 3.5
        ))           

KR_score = rmsle_cv(KR)
print("Kernel Ridge score: {:.4f} ({:.4f})".format(KR_score.mean(), KR_score.std()))

Kernel Ridge score: 0.1191 (0.0152)


### Elastic Net

In [8]:
ENet = make_pipeline(RobustScaler(), ElasticNet(
        alpha= 0.0002835,
        l1_ratio= 0.9999905,
        random_state= 0 
        ))

ENet_score = rmsle_cv(ENet)
print("ENet score: {:.4f} ({:.4f})".format(ENet_score.mean(), ENet_score.std()))

ENet score: 0.1130 (0.0136)


### Lasso

In [9]:
Lass = make_pipeline(RobustScaler(), Lasso(
        max_iter=3000,
        alpha= 0.000281,
        random_state= 5 
        )) 

Lass_score = rmsle_cv(Lass)
print("Lass score: {:.4f} ({:.4f})".format(Lass_score.mean(), Lass_score.std()))


Lass score: 0.1130 (0.0136)


### Gradient Boosting

In [10]:
Graboost = GradientBoostingRegressor(
    n_estimators= 3000,
    max_depth= 4,
    min_samples_leaf= 2,
    max_features= 'sqrt',
    loss='huber',
    random_state= 0,
    learning_rate= 0.0223,
    min_samples_split= 100 
    )

Graboost_score = rmsle_cv(Graboost)
print("Graboost score: {:.4f} ({:.4f})".format(Graboost_score.mean(), Graboost_score.std()))

Graboost score: 0.1097 (0.0169)


### Ridge

In [11]:
RidgeR = make_pipeline(RobustScaler(), Ridge(
    alpha= 8.8435,
    solver= 'cholesky'
        ))    
                       
RidgeR_score = rmsle_cv(RidgeR)
print("RidgeR score: {:.4f} ({:.4f})".format(RidgeR_score.mean(), RidgeR_score.std()))

RidgeR score: 0.1151 (0.0133)


### XGBoost

In [12]:
XGBoost = xgb.XGBRegressor(
    learning_rate= 0.0225,
    colsample_bytree= 0.1,
    n_estimators= 2500,
    max_depth= 3,
    subsample= 0.6982,
    min_child_weight=1, 
    )

XGBoost_score = rmsle_cv(XGBoost)
print("XGBoost score: {:.4f} ({:.4f})".format(XGBoost_score.mean(), XGBoost_score.std()))

XGBoost score: 0.1094 (0.0159)


### Bayesian Ridge

In [13]:
BRidge = make_pipeline(RobustScaler(), BayesianRidge(
        n_iter= 100,
        alpha_1= 6e-05,
        alpha_2= 0.01,
        lambda_1= 0.01,
        lambda_2= 6e-05,
        tol= 1e-05
        ))
                       
BRidge_score = rmsle_cv(BRidge)
print("BRidge score: {:.4f} ({:.4f})".format(BRidge_score.mean(), BRidge_score.std()))

BRidge score: 0.1152 (0.0133)


### LightGBM

In [14]:
LGB = lgb.LGBMRegressor(
    learning_rate= 0.0166,
    objective= 'regression',
    num_leaves= 30,
    num_iterations= 1500,
    feature_fraction = 0.2175,
    bagging_fraction= .9870,
    min_data_in_leaf= 9,
    max_depth= 68
)

LGB_score = rmsle_cv(LGB)
print("LGB score: {:.4f} ({:.4f})".format(LGB_score.mean(), LGB_score.std()))

LGB score: 0.1175 (0.0165)


### CatBoost

In [15]:
CatB = make_pipeline(RobustScaler(), CatBoostRegressor(
    iterations= 1929,
    depth= 2, 
    learning_rate= 0.0229,
    loss_function= 'RMSE',
    l2_leaf_reg=0,
    rsm= 1.0,
    one_hot_max_size=194
))

CatB_score = rmsle_cv(CatB)
print("CatB score: {:.4f} ({:.4f})".format(CatB_score.mean(), CatB_score.std()))

0:	learn: 11.7582666	total: 50.2ms	remaining: 1m 36s
1:	learn: 11.4893127	total: 52.6ms	remaining: 50.7s
2:	learn: 11.2264961	total: 55.4ms	remaining: 35.5s
3:	learn: 10.9697377	total: 58.6ms	remaining: 28.2s
4:	learn: 10.7188648	total: 61.1ms	remaining: 23.5s
5:	learn: 10.4737473	total: 64ms	remaining: 20.5s
6:	learn: 10.2342115	total: 66.5ms	remaining: 18.3s
7:	learn: 10.0001037	total: 69ms	remaining: 16.6s
8:	learn: 9.7714469	total: 71ms	remaining: 15.2s
9:	learn: 9.5479483	total: 73.2ms	remaining: 14.1s
10:	learn: 9.3296748	total: 75.2ms	remaining: 13.1s
11:	learn: 9.1164044	total: 76.8ms	remaining: 12.3s
12:	learn: 8.9080139	total: 78.4ms	remaining: 11.6s
13:	learn: 8.7044173	total: 80.1ms	remaining: 11s
14:	learn: 8.5054963	total: 81.5ms	remaining: 10.4s
15:	learn: 8.3110940	total: 82.8ms	remaining: 9.9s
16:	learn: 8.1211814	total: 85ms	remaining: 9.56s
17:	learn: 7.9356424	total: 86.5ms	remaining: 9.18s
18:	learn: 7.7543595	total: 87.9ms	remaining: 8.84s
19:	learn: 7.5772403	tot

164:	learn: 0.3606086	total: 471ms	remaining: 5.04s
165:	learn: 0.3555137	total: 474ms	remaining: 5.04s
166:	learn: 0.3501492	total: 477ms	remaining: 5.04s
167:	learn: 0.3446026	total: 480ms	remaining: 5.03s
168:	learn: 0.3393741	total: 483ms	remaining: 5.03s
169:	learn: 0.3346073	total: 485ms	remaining: 5.02s
170:	learn: 0.3299689	total: 488ms	remaining: 5.01s
171:	learn: 0.3251744	total: 490ms	remaining: 5s
172:	learn: 0.3205644	total: 492ms	remaining: 4.99s
173:	learn: 0.3158195	total: 494ms	remaining: 4.98s
174:	learn: 0.3117318	total: 495ms	remaining: 4.97s
175:	learn: 0.3074586	total: 497ms	remaining: 4.95s
176:	learn: 0.3032791	total: 499ms	remaining: 4.93s
177:	learn: 0.2995982	total: 500ms	remaining: 4.92s
178:	learn: 0.2955434	total: 501ms	remaining: 4.9s
179:	learn: 0.2921451	total: 503ms	remaining: 4.88s
180:	learn: 0.2885840	total: 510ms	remaining: 4.92s
181:	learn: 0.2846162	total: 518ms	remaining: 4.97s
182:	learn: 0.2810660	total: 523ms	remaining: 4.99s
183:	learn: 0.27

368:	learn: 0.1306090	total: 1.02s	remaining: 4.31s
369:	learn: 0.1304054	total: 1.02s	remaining: 4.3s
370:	learn: 0.1303074	total: 1.02s	remaining: 4.3s
371:	learn: 0.1302031	total: 1.02s	remaining: 4.29s
372:	learn: 0.1301194	total: 1.03s	remaining: 4.28s
373:	learn: 0.1300702	total: 1.03s	remaining: 4.27s
374:	learn: 0.1299504	total: 1.03s	remaining: 4.27s
375:	learn: 0.1299374	total: 1.03s	remaining: 4.26s
376:	learn: 0.1298754	total: 1.03s	remaining: 4.25s
377:	learn: 0.1296089	total: 1.03s	remaining: 4.24s
378:	learn: 0.1294758	total: 1.03s	remaining: 4.24s
379:	learn: 0.1293529	total: 1.04s	remaining: 4.24s
380:	learn: 0.1291680	total: 1.05s	remaining: 4.27s
381:	learn: 0.1289935	total: 1.06s	remaining: 4.28s
382:	learn: 0.1287865	total: 1.06s	remaining: 4.28s
383:	learn: 0.1287345	total: 1.06s	remaining: 4.28s
384:	learn: 0.1286058	total: 1.07s	remaining: 4.28s
385:	learn: 0.1284987	total: 1.07s	remaining: 4.28s
386:	learn: 0.1284698	total: 1.07s	remaining: 4.28s
387:	learn: 0.

532:	learn: 0.1185142	total: 1.41s	remaining: 3.68s
533:	learn: 0.1184979	total: 1.41s	remaining: 3.68s
534:	learn: 0.1184878	total: 1.41s	remaining: 3.67s
535:	learn: 0.1184770	total: 1.41s	remaining: 3.67s
536:	learn: 0.1183829	total: 1.41s	remaining: 3.66s
537:	learn: 0.1183144	total: 1.42s	remaining: 3.66s
538:	learn: 0.1182476	total: 1.42s	remaining: 3.65s
539:	learn: 0.1182456	total: 1.42s	remaining: 3.65s
540:	learn: 0.1181562	total: 1.42s	remaining: 3.64s
541:	learn: 0.1181510	total: 1.42s	remaining: 3.64s
542:	learn: 0.1180921	total: 1.42s	remaining: 3.63s
543:	learn: 0.1180042	total: 1.43s	remaining: 3.64s
544:	learn: 0.1179921	total: 1.44s	remaining: 3.66s
545:	learn: 0.1178718	total: 1.44s	remaining: 3.66s
546:	learn: 0.1178658	total: 1.45s	remaining: 3.66s
547:	learn: 0.1178612	total: 1.45s	remaining: 3.66s
548:	learn: 0.1178568	total: 1.46s	remaining: 3.66s
549:	learn: 0.1178565	total: 1.46s	remaining: 3.66s
550:	learn: 0.1178482	total: 1.46s	remaining: 3.66s
551:	learn: 

694:	learn: 0.1136310	total: 1.83s	remaining: 3.24s
695:	learn: 0.1136295	total: 1.83s	remaining: 3.25s
696:	learn: 0.1136207	total: 1.83s	remaining: 3.25s
697:	learn: 0.1136104	total: 1.84s	remaining: 3.24s
698:	learn: 0.1135667	total: 1.84s	remaining: 3.24s
699:	learn: 0.1135623	total: 1.85s	remaining: 3.24s
700:	learn: 0.1135426	total: 1.85s	remaining: 3.24s
701:	learn: 0.1135382	total: 1.85s	remaining: 3.24s
702:	learn: 0.1135136	total: 1.85s	remaining: 3.23s
703:	learn: 0.1134764	total: 1.86s	remaining: 3.24s
704:	learn: 0.1134663	total: 1.86s	remaining: 3.23s
705:	learn: 0.1134553	total: 1.86s	remaining: 3.23s
706:	learn: 0.1134457	total: 1.86s	remaining: 3.22s
707:	learn: 0.1134412	total: 1.87s	remaining: 3.22s
708:	learn: 0.1134295	total: 1.87s	remaining: 3.22s
709:	learn: 0.1134228	total: 1.87s	remaining: 3.21s
710:	learn: 0.1134048	total: 1.87s	remaining: 3.21s
711:	learn: 0.1133821	total: 1.87s	remaining: 3.2s
712:	learn: 0.1133754	total: 1.88s	remaining: 3.2s
713:	learn: 0.

...

1127:	learn: 0.1066122	total: 2.68s	remaining: 1.9s
1128:	learn: 0.1066121	total: 2.68s	remaining: 1.9s
1129:	learn: 0.1066070	total: 2.68s	remaining: 1.9s
1130:	learn: 0.1065869	total: 2.69s	remaining: 1.9s
1131:	learn: 0.1065868	total: 2.7s	remaining: 1.9s
1132:	learn: 0.1065857	total: 2.71s	remaining: 1.9s
1133:	learn: 0.1065791	total: 2.71s	remaining: 1.9s
1134:	learn: 0.1065769	total: 2.71s	remaining: 1.9s
1135:	learn: 0.1065752	total: 2.72s	remaining: 1.9s
1136:	learn: 0.1065664	total: 2.72s	remaining: 1.9s
1137:	learn: 0.1065386	total: 2.72s	remaining: 1.89s
1138:	learn: 0.1065386	total: 2.73s	remaining: 1.89s
1139:	learn: 0.1065385	total: 2.73s	remaining: 1.89s
1140:	learn: 0.1065016	total: 2.73s	remaining: 1.89s
1141:	learn: 0.1065011	total: 2.73s	remaining: 1.88s
1142:	learn: 0.1064988	total: 2.74s	remaining: 1.88s
1143:	learn: 0.1064645	total: 2.74s	remaining: 1.88s
1144:	learn: 0.1064621	total: 2.74s	remaining: 1.88s
1145:	learn: 0.1064335	total: 2.74s	remaining: 1.87s
1146

1284:	learn: 0.1044333	total: 3.06s	remaining: 1.53s
1285:	learn: 0.1044172	total: 3.07s	remaining: 1.53s
1286:	learn: 0.1044149	total: 3.07s	remaining: 1.53s
1287:	learn: 0.1043684	total: 3.07s	remaining: 1.53s
1288:	learn: 0.1043607	total: 3.07s	remaining: 1.53s
1289:	learn: 0.1043607	total: 3.08s	remaining: 1.52s
1290:	learn: 0.1043186	total: 3.08s	remaining: 1.52s
1291:	learn: 0.1043084	total: 3.08s	remaining: 1.52s
1292:	learn: 0.1043050	total: 3.08s	remaining: 1.52s
1293:	learn: 0.1043050	total: 3.09s	remaining: 1.52s
1294:	learn: 0.1043030	total: 3.1s	remaining: 1.52s
1295:	learn: 0.1042872	total: 3.1s	remaining: 1.51s
1296:	learn: 0.1042183	total: 3.1s	remaining: 1.51s
1297:	learn: 0.1042168	total: 3.11s	remaining: 1.51s
1298:	learn: 0.1042109	total: 3.11s	remaining: 1.51s
1299:	learn: 0.1042068	total: 3.11s	remaining: 1.51s
1300:	learn: 0.1042032	total: 3.11s	remaining: 1.5s
1301:	learn: 0.1041996	total: 3.12s	remaining: 1.5s
1302:	learn: 0.1041996	total: 3.12s	remaining: 1.5s

1445:	learn: 0.1029006	total: 3.44s	remaining: 1.15s
1446:	learn: 0.1028978	total: 3.44s	remaining: 1.15s
1447:	learn: 0.1028875	total: 3.44s	remaining: 1.14s
1448:	learn: 0.1028875	total: 3.45s	remaining: 1.14s
1449:	learn: 0.1028442	total: 3.45s	remaining: 1.14s
1450:	learn: 0.1028430	total: 3.45s	remaining: 1.14s
1451:	learn: 0.1028430	total: 3.45s	remaining: 1.13s
1452:	learn: 0.1028059	total: 3.45s	remaining: 1.13s
1453:	learn: 0.1028024	total: 3.46s	remaining: 1.13s
1454:	learn: 0.1027966	total: 3.47s	remaining: 1.13s
1455:	learn: 0.1027965	total: 3.48s	remaining: 1.13s
1456:	learn: 0.1027928	total: 3.48s	remaining: 1.13s
1457:	learn: 0.1027928	total: 3.48s	remaining: 1.13s
1458:	learn: 0.1027928	total: 3.48s	remaining: 1.12s
1459:	learn: 0.1027782	total: 3.49s	remaining: 1.12s
1460:	learn: 0.1027734	total: 3.49s	remaining: 1.12s
1461:	learn: 0.1027090	total: 3.49s	remaining: 1.11s
1462:	learn: 0.1026945	total: 3.5s	remaining: 1.11s
1463:	learn: 0.1026943	total: 3.5s	remaining: 1

1604:	learn: 0.1015899	total: 3.81s	remaining: 770ms
1605:	learn: 0.1015718	total: 3.82s	remaining: 768ms
1606:	learn: 0.1015694	total: 3.82s	remaining: 765ms
1607:	learn: 0.1015485	total: 3.82s	remaining: 763ms
1608:	learn: 0.1015484	total: 3.82s	remaining: 760ms
1609:	learn: 0.1015018	total: 3.82s	remaining: 758ms
1610:	learn: 0.1014872	total: 3.82s	remaining: 755ms
1611:	learn: 0.1014789	total: 3.83s	remaining: 752ms
1612:	learn: 0.1014747	total: 3.83s	remaining: 750ms
1613:	learn: 0.1014743	total: 3.83s	remaining: 747ms
1614:	learn: 0.1014741	total: 3.83s	remaining: 746ms
1615:	learn: 0.1014658	total: 3.85s	remaining: 745ms
1616:	learn: 0.1014656	total: 3.85s	remaining: 743ms
1617:	learn: 0.1014655	total: 3.85s	remaining: 740ms
1618:	learn: 0.1014655	total: 3.85s	remaining: 738ms
1619:	learn: 0.1014521	total: 3.86s	remaining: 736ms
1620:	learn: 0.1014519	total: 3.86s	remaining: 734ms
1621:	learn: 0.1014338	total: 3.86s	remaining: 731ms
1622:	learn: 0.1014338	total: 3.86s	remaining:

1775:	learn: 0.1004649	total: 4.2s	remaining: 362ms
1776:	learn: 0.1004645	total: 4.2s	remaining: 359ms
1777:	learn: 0.1004644	total: 4.2s	remaining: 357ms
1778:	learn: 0.1004643	total: 4.2s	remaining: 354ms
1779:	learn: 0.1004399	total: 4.2s	remaining: 352ms
1780:	learn: 0.1004297	total: 4.21s	remaining: 350ms
1781:	learn: 0.1003353	total: 4.22s	remaining: 348ms
1782:	learn: 0.1003351	total: 4.22s	remaining: 346ms
1783:	learn: 0.1003334	total: 4.23s	remaining: 344ms
1784:	learn: 0.1003303	total: 4.23s	remaining: 341ms
1785:	learn: 0.1003288	total: 4.24s	remaining: 339ms
1786:	learn: 0.1003273	total: 4.24s	remaining: 337ms
1787:	learn: 0.1003273	total: 4.24s	remaining: 334ms
1788:	learn: 0.1002998	total: 4.24s	remaining: 332ms
1789:	learn: 0.1002981	total: 4.25s	remaining: 330ms
1790:	learn: 0.1002813	total: 4.25s	remaining: 327ms
1791:	learn: 0.1002812	total: 4.25s	remaining: 325ms
1792:	learn: 0.1002808	total: 4.25s	remaining: 322ms
1793:	learn: 0.1002764	total: 4.25s	remaining: 320m

### Stacking

In [16]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    #cloning models to fit the data
    def fit(self, X_train, Y_train):
        self.models_ = [clone(x) for x in self.models]
        
        #training the cloned models
        for model in self.models_:
            model.fit(X_train.values, Y_train)
            
        return self
    
    #predicting based on the cloned models
    def predict(self, X_val):
        predictions = np.column_stack([
            model.predict(X_val) for model in self.models_])
        #averaging the predictions
        return np.mean(predictions, axis=1)

### Averaged  models score

In [17]:
# stack models together
averaged_models = AveragingModels(models = (ENet, Lass, Graboost, RidgeR))

In [18]:
# compute score of stacked models
score = rmsle_cv(averaged_models)
print(" Averaged models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged models score: 0.1094 (0.0145)



In [19]:
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

### Stacked Model Prediction Score

In [20]:
averaged_models.fit(X_train, Y_train)
averaged_train_pred = averaged_models.predict(X_val)
# stacked_predict = np.exp(averaged_train_pred)
print(rmse(Y_val, averaged_train_pred))

0.11569591056976032


### XGBoost Model Prediction Score

In [21]:
XGBoost.fit(X_train, Y_train)
XGBoost_train_predict = XGBoost.predict(X_val)
# XGBoost_predict = np.exp(XGBoost_train_predict)
print(rmse(Y_val, XGBoost_train_predict))

0.12042504230217307


### Light GBM Model Prediction Score

In [22]:
LGB.fit(X_train, Y_train)
LGB_train_predict = LGB.predict(X_val)
# LGB_predict = np.exp(LGB_train_predict)
rmse(Y_val, LGB_train_predict)

0.1310220674403337

### CatB Model Prediction Score

In [23]:
CatB.fit(X_train, Y_train)
CatB_train_predict = CatB.predict(X_val)
# XGBoost_predict = np.exp(CatB_train_predict)
print(rmse(Y_val, CatB_train_predict))

0:	learn: 11.7547183	total: 1.96ms	remaining: 3.79s
1:	learn: 11.4858211	total: 4.16ms	remaining: 4.01s
2:	learn: 11.2230571	total: 6.12ms	remaining: 3.93s
3:	learn: 10.9663549	total: 7.56ms	remaining: 3.64s
4:	learn: 10.7155454	total: 9.23ms	remaining: 3.55s
5:	learn: 10.4704393	total: 10.6ms	remaining: 3.38s
6:	learn: 10.2309437	total: 12ms	remaining: 3.29s
7:	learn: 9.9969963	total: 13.4ms	remaining: 3.22s
8:	learn: 9.7684170	total: 14.8ms	remaining: 3.17s
9:	learn: 9.5450810	total: 16.2ms	remaining: 3.1s
10:	learn: 9.3268263	total: 17.6ms	remaining: 3.06s
11:	learn: 9.1136172	total: 18.7ms	remaining: 2.98s
12:	learn: 8.9052934	total: 23ms	remaining: 3.38s
13:	learn: 8.7017558	total: 30.4ms	remaining: 4.15s
14:	learn: 8.5028043	total: 34.6ms	remaining: 4.41s
15:	learn: 8.3084988	total: 37.9ms	remaining: 4.53s
16:	learn: 8.1186486	total: 40.5ms	remaining: 4.55s
17:	learn: 7.9331483	total: 43ms	remaining: 4.57s
18:	learn: 7.7518552	total: 45.2ms	remaining: 4.54s
19:	learn: 7.5747729	t

235:	learn: 0.1821111	total: 584ms	remaining: 4.19s
236:	learn: 0.1813921	total: 586ms	remaining: 4.18s
237:	learn: 0.1805456	total: 588ms	remaining: 4.18s
238:	learn: 0.1798529	total: 590ms	remaining: 4.17s
239:	learn: 0.1789761	total: 592ms	remaining: 4.16s
240:	learn: 0.1782397	total: 593ms	remaining: 4.15s
241:	learn: 0.1774862	total: 595ms	remaining: 4.15s
242:	learn: 0.1765658	total: 597ms	remaining: 4.14s
243:	learn: 0.1757727	total: 598ms	remaining: 4.13s
244:	learn: 0.1750912	total: 600ms	remaining: 4.13s
245:	learn: 0.1742965	total: 611ms	remaining: 4.18s
246:	learn: 0.1736318	total: 617ms	remaining: 4.2s
247:	learn: 0.1728946	total: 622ms	remaining: 4.21s
248:	learn: 0.1721759	total: 627ms	remaining: 4.23s
249:	learn: 0.1717589	total: 632ms	remaining: 4.25s
250:	learn: 0.1710969	total: 637ms	remaining: 4.25s
251:	learn: 0.1704198	total: 640ms	remaining: 4.26s
252:	learn: 0.1695910	total: 643ms	remaining: 4.26s
253:	learn: 0.1689783	total: 646ms	remaining: 4.26s
254:	learn: 0

396:	learn: 0.1325545	total: 970ms	remaining: 3.74s
397:	learn: 0.1324057	total: 972ms	remaining: 3.74s
398:	learn: 0.1322959	total: 974ms	remaining: 3.74s
399:	learn: 0.1321931	total: 976ms	remaining: 3.73s
400:	learn: 0.1321072	total: 978ms	remaining: 3.73s
401:	learn: 0.1320307	total: 979ms	remaining: 3.72s
402:	learn: 0.1319967	total: 981ms	remaining: 3.71s
403:	learn: 0.1319119	total: 982ms	remaining: 3.71s
404:	learn: 0.1318798	total: 984ms	remaining: 3.7s
405:	learn: 0.1316404	total: 991ms	remaining: 3.72s
406:	learn: 0.1315066	total: 999ms	remaining: 3.73s
407:	learn: 0.1315032	total: 1s	remaining: 3.74s
408:	learn: 0.1314431	total: 1.01s	remaining: 3.75s
409:	learn: 0.1312875	total: 1.01s	remaining: 3.75s
410:	learn: 0.1312133	total: 1.01s	remaining: 3.75s
411:	learn: 0.1309800	total: 1.02s	remaining: 3.75s
412:	learn: 0.1308071	total: 1.02s	remaining: 3.75s
413:	learn: 0.1308064	total: 1.02s	remaining: 3.74s
414:	learn: 0.1307426	total: 1.02s	remaining: 3.74s
415:	learn: 0.13

564:	learn: 0.1225114	total: 1.35s	remaining: 3.26s
565:	learn: 0.1224814	total: 1.35s	remaining: 3.26s
566:	learn: 0.1224299	total: 1.35s	remaining: 3.25s
567:	learn: 0.1223769	total: 1.36s	remaining: 3.25s
568:	learn: 0.1223351	total: 1.36s	remaining: 3.26s
569:	learn: 0.1221881	total: 1.37s	remaining: 3.27s
570:	learn: 0.1221720	total: 1.38s	remaining: 3.27s
571:	learn: 0.1221706	total: 1.38s	remaining: 3.27s
572:	learn: 0.1221691	total: 1.38s	remaining: 3.27s
573:	learn: 0.1221677	total: 1.39s	remaining: 3.27s
574:	learn: 0.1221664	total: 1.39s	remaining: 3.27s
575:	learn: 0.1221643	total: 1.39s	remaining: 3.27s
576:	learn: 0.1221507	total: 1.4s	remaining: 3.27s
577:	learn: 0.1221105	total: 1.4s	remaining: 3.27s
578:	learn: 0.1221087	total: 1.4s	remaining: 3.27s
579:	learn: 0.1219799	total: 1.4s	remaining: 3.27s
580:	learn: 0.1219675	total: 1.41s	remaining: 3.26s
581:	learn: 0.1219657	total: 1.41s	remaining: 3.26s
582:	learn: 0.1219637	total: 1.41s	remaining: 3.25s
583:	learn: 0.12

733:	learn: 0.1175984	total: 1.75s	remaining: 2.85s
734:	learn: 0.1175889	total: 1.76s	remaining: 2.85s
735:	learn: 0.1175575	total: 1.76s	remaining: 2.86s
736:	learn: 0.1175480	total: 1.76s	remaining: 2.85s
737:	learn: 0.1175114	total: 1.77s	remaining: 2.86s
738:	learn: 0.1175110	total: 1.77s	remaining: 2.85s
739:	learn: 0.1175050	total: 1.77s	remaining: 2.85s
740:	learn: 0.1174979	total: 1.78s	remaining: 2.85s
741:	learn: 0.1174933	total: 1.78s	remaining: 2.85s
742:	learn: 0.1174082	total: 1.78s	remaining: 2.85s
743:	learn: 0.1174059	total: 1.78s	remaining: 2.84s
744:	learn: 0.1173962	total: 1.79s	remaining: 2.84s
745:	learn: 0.1173668	total: 1.79s	remaining: 2.84s
746:	learn: 0.1173146	total: 1.79s	remaining: 2.83s
747:	learn: 0.1173079	total: 1.79s	remaining: 2.83s
748:	learn: 0.1173060	total: 1.79s	remaining: 2.83s
749:	learn: 0.1173015	total: 1.8s	remaining: 2.82s
750:	learn: 0.1172165	total: 1.8s	remaining: 2.82s
751:	learn: 0.1172138	total: 1.8s	remaining: 2.82s
752:	learn: 0.1

...

1231:	learn: 0.1097892	total: 2.89s	remaining: 1.64s
1232:	learn: 0.1097807	total: 2.9s	remaining: 1.63s
1233:	learn: 0.1097292	total: 2.9s	remaining: 1.63s
1234:	learn: 0.1097166	total: 2.9s	remaining: 1.63s
1235:	learn: 0.1097164	total: 2.9s	remaining: 1.63s
1236:	learn: 0.1097116	total: 2.91s	remaining: 1.63s
1237:	learn: 0.1097092	total: 2.91s	remaining: 1.62s
1238:	learn: 0.1097084	total: 2.91s	remaining: 1.62s
1239:	learn: 0.1096938	total: 2.92s	remaining: 1.62s
1240:	learn: 0.1096935	total: 2.92s	remaining: 1.62s
1241:	learn: 0.1096930	total: 2.92s	remaining: 1.61s
1242:	learn: 0.1096921	total: 2.92s	remaining: 1.61s
1243:	learn: 0.1096800	total: 2.92s	remaining: 1.61s
1244:	learn: 0.1096796	total: 2.92s	remaining: 1.61s
1245:	learn: 0.1096749	total: 2.93s	remaining: 1.6s
1246:	learn: 0.1096743	total: 2.93s	remaining: 1.6s
1247:	learn: 0.1096741	total: 2.93s	remaining: 1.6s
1248:	learn: 0.1096734	total: 2.93s	remaining: 1.59s
1249:	learn: 0.1096704	total: 2.93s	remaining: 1.59s


1401:	learn: 0.1084946	total: 3.27s	remaining: 1.23s
1402:	learn: 0.1084833	total: 3.27s	remaining: 1.23s
1403:	learn: 0.1084666	total: 3.28s	remaining: 1.23s
1404:	learn: 0.1084477	total: 3.28s	remaining: 1.22s
1405:	learn: 0.1084246	total: 3.28s	remaining: 1.22s
1406:	learn: 0.1084113	total: 3.28s	remaining: 1.22s
1407:	learn: 0.1083519	total: 3.29s	remaining: 1.22s
1408:	learn: 0.1083500	total: 3.29s	remaining: 1.21s
1409:	learn: 0.1083496	total: 3.29s	remaining: 1.21s
1410:	learn: 0.1083471	total: 3.29s	remaining: 1.21s
1411:	learn: 0.1083449	total: 3.3s	remaining: 1.21s
1412:	learn: 0.1083331	total: 3.3s	remaining: 1.2s
1413:	learn: 0.1083317	total: 3.3s	remaining: 1.2s
1414:	learn: 0.1083226	total: 3.3s	remaining: 1.2s
1415:	learn: 0.1083220	total: 3.3s	remaining: 1.2s
1416:	learn: 0.1083219	total: 3.31s	remaining: 1.19s
1417:	learn: 0.1083217	total: 3.31s	remaining: 1.19s
1418:	learn: 0.1083213	total: 3.31s	remaining: 1.19s
1419:	learn: 0.1082917	total: 3.31s	remaining: 1.19s
14

1559:	learn: 0.1076333	total: 3.64s	remaining: 861ms
1560:	learn: 0.1076327	total: 3.65s	remaining: 860ms
1561:	learn: 0.1076326	total: 3.65s	remaining: 858ms
1562:	learn: 0.1076324	total: 3.65s	remaining: 856ms
1563:	learn: 0.1076323	total: 3.66s	remaining: 854ms
1564:	learn: 0.1076322	total: 3.66s	remaining: 851ms
1565:	learn: 0.1076283	total: 3.66s	remaining: 849ms
1566:	learn: 0.1076283	total: 3.67s	remaining: 847ms
1567:	learn: 0.1076282	total: 3.67s	remaining: 845ms
1568:	learn: 0.1076238	total: 3.67s	remaining: 842ms
1569:	learn: 0.1076229	total: 3.67s	remaining: 840ms
1570:	learn: 0.1076099	total: 3.68s	remaining: 838ms
1571:	learn: 0.1076095	total: 3.68s	remaining: 836ms
1572:	learn: 0.1076088	total: 3.68s	remaining: 833ms
1573:	learn: 0.1076086	total: 3.68s	remaining: 830ms
1574:	learn: 0.1076080	total: 3.68s	remaining: 828ms
1575:	learn: 0.1076075	total: 3.69s	remaining: 825ms
1576:	learn: 0.1075786	total: 3.69s	remaining: 823ms
1577:	learn: 0.1075784	total: 3.69s	remaining:

1715:	learn: 0.1071029	total: 4.01s	remaining: 498ms
1716:	learn: 0.1070996	total: 4.02s	remaining: 496ms
1717:	learn: 0.1070966	total: 4.02s	remaining: 494ms
1718:	learn: 0.1070964	total: 4.02s	remaining: 492ms
1719:	learn: 0.1070957	total: 4.03s	remaining: 489ms
1720:	learn: 0.1070950	total: 4.03s	remaining: 487ms
1721:	learn: 0.1070944	total: 4.03s	remaining: 485ms
1722:	learn: 0.1070326	total: 4.04s	remaining: 483ms
1723:	learn: 0.1070324	total: 4.04s	remaining: 480ms
1724:	learn: 0.1070310	total: 4.04s	remaining: 478ms
1725:	learn: 0.1070046	total: 4.04s	remaining: 476ms
1726:	learn: 0.1069993	total: 4.05s	remaining: 473ms
1727:	learn: 0.1069986	total: 4.05s	remaining: 471ms
1728:	learn: 0.1069985	total: 4.05s	remaining: 469ms
1729:	learn: 0.1069978	total: 4.05s	remaining: 466ms
1730:	learn: 0.1069977	total: 4.05s	remaining: 464ms
1731:	learn: 0.1069975	total: 4.05s	remaining: 461ms
1732:	learn: 0.1069247	total: 4.06s	remaining: 459ms
1733:	learn: 0.1068641	total: 4.06s	remaining:

0.12998356848232873


### RMSE on the entire training set

In [30]:
overall_stack = XGBoost_train_predict * 0.3 + LGB_train_predict * 0.1 + CatB_train_predict * 0.1 + averaged_train_pred * 0.5

In [31]:
print( "RMSE on the entire training dataset: {}".format(
    rmse(Y_val, overall_stack)
))

RMSE on the entire training dataset: 0.11576681180144405
